# Fast ICA & the independent components of image patches

## Paola Suaréz, Elisabeth Kress, Esra Zihni, Jiameng Wu

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from sklearn.decomposition import FastICA
from time import time
from sklearn.feature_extraction.image import extract_patches_2d

%matplotlib inline

### 7.1 Negentropy is scale-invariant

### 7.2 fastICA vs. Infomax

In [2]:
# Helper function for the transformed data
def logistic(y):
    return 1/(1+np.exp(-y))

# Helper fucntion for invertible matrix
def invert(N=3):
    #matrix = np.random.normal(loc=mu, scale=sigma, size=(N, N)) #scipy.stats.ortho_group.rvs
    matrix = np.random.uniform(0, 1, (N, N))
    exception = True
    while exception:
        try:
            np.linalg.inv(matrix)
        except:
            matrix = np.random.uniform(0, 1, (N, N))
        else:
            exception = False
    return matrix

# natural gradient ascent
def nat_meng(data, w0, eps=0.01, lamb=0.9999):
    """
    Computes natural gradient ascent on an array of mixed sounds in order to obtain the best weight matrix.
    data: an array containing mixed sources.
    w0: initial weight matrix, randomly generated.
    returns
    ws: an array of all the evolution of weights.
    """
#    ws = np.zeros((data.shape[1]+1, data.shape[0], data.shape[0]))
#     ws[0] = w0
    dim = data.shape[0]
    length = data.shape[1]
    W = w0
    for t in np.arange(length):
        x = data[:, t]
#        W = ws[t].copy()
        eps = eps * lamb
        #eps = eps0/(t+1)
        delta = np.identity(dim)
        u = np.dot(W, x)
        update = np.dot(delta, W) + np.dot((1- 2*logistic(u)).reshape(dim,1), np.dot(u.reshape(1,dim), W))
        delta_W = eps * update
        W += delta_W
#        ws[t+1] = W
    return W

In [3]:
# Load sounds
source1 = np.loadtxt('sound1.dat')
source2 = np.loadtxt('sound2.dat')
original = np.array([source1, source2])

In [4]:
# Create an invertible mixing matrix
A = invert(2)

# Mix the sources
mixed = np.dot(A, original)

# Center the data
mixed_c = np.subtract(mixed.T, mixed.mean(axis=1)).T

# Initialize weights
W0 = invert(2)

In [5]:
start1 = time()

# Run natural gradient
W = nat_meng(mixed_c, W0)

# Unmixed
# W = Ws[-1]
unmixed = np.dot(W, mixed_c)

end1 = time()

In [6]:
start2 = time()

# Run FastICA
ica = FastICA(2)
S_ = ica.fit_transform(mixed.T).T
A_ = ica.mixing_

end2 = time()

In [7]:
nat_time = end1-start1
fast_time = end2-start2
print(nat_time)
print(fast_time)

0.4390747547149658
0.011027097702026367


In [8]:
Audio(unmixed[0], rate=8192)

In [9]:
Audio(S_[1], rate=8192)

### 7.3 ICA on Image Patches

In [10]:
# Read images
nature = []   # 13 nature images
for i in range(13):
    nature.append(plt.imread("imgpca/n"+str(i+1)+".jpg", format='jpg'))

buildings = []   # 10 building images, without considering the zooms of the 9th img.
for i in range(10):
    buildings.append(plt.imread("imgpca/b"+str(i+1)+".jpg", format='jpg'))
    
text = []   # 14 text images
for i in range(14):
    text.append(plt.imread("imgpca/t"+str(i+1)+".jpg", format='jpg'))

In [25]:
# Patches
P = 20000
N = 144
pix = int(np.sqrt(N))

n_patches = np.zeros((len(nature), P, N))
b_patches = np.zeros((len(buildings), P, N))
t_patches = np.zeros((len(text), P, N))

images_ = [nature, buildings, text]
patches_ = [n_patches, b_patches, t_patches]

for i in range(3):
    images = images_[i]
    patches = patches_[i]
    for j in range(len(image)):
        patches[i,:,:] = extract_patches_2d(images[i], (pix, pix), max_patches=P).reshape(P,N)

In [26]:
# Calculating independent features of the images using FastICA
# Standard setting of the FastICA function is whitening and contrast function logcosh a=1

reconstructed = 
for i in range(3):
    image = images_[i][0]
    size = image.shape[0] * image.shape[1]
    patches = patches_[i]  

    for j in range(patches.shape[0]):
        img = patches[j]
        ica = FastICA(size)
        S_ = ica.fit_transform(img)


(13, 20000, 144)
(10, 20000, 144)
(14, 20000, 144)
